In [1]:
#viabiliza imports do root dir do proj
import os
import sys
proj_dir = os.path.split(os.path.abspath('..'))[0]
print(proj_dir)
if proj_dir not in sys.path:
    sys.path.append(proj_dir)

/home/h-pgy/Desktop/Projetos/middleware_orcamento_smae


In [2]:
import requests
from core.utils import run_server

In [3]:
def get_request(endpoint:str, base_url = 'http://localhost', **params)->dict:
    
    url = base_url + endpoint
    
    if params:
        params = [f'{key}={val}' for key, val in params.items()]
        params = '&'.join(params)
        url = url + '?' + params
        
    with requests.get(url) as r:
        return r.json()

In [4]:
def post_request(endpoint:str, data:dict, base_url = 'http://localhost',
                 **params)->dict:
    
    url = base_url + endpoint
    
    if params:
        params = [f'{key}={val}' for key, val in params.items()]
        params = '&'.join(params)
        url = url + '?' + params
        
    with requests.post(url, json=data, 
                       headers={'Content-Type': 'application/json',
                               'accept': 'application/json'}) as r:

        return r.json()

Checando se subiu o servidor

In [5]:
def subir_servidor_se_nao_up():
    
    try:
        assert get_request('/')['info']['title'] == 'Middleware_orcamento'
    except:
        run_server()

In [6]:
subir_servidor_se_nao_up()

In [7]:
assert get_request('/')['info']['title'] == 'Middleware_orcamento'

In [8]:
swagger = get_request('')

Listando endpoints

In [9]:
swagger['paths'].keys()

dict_keys(['/v1/empenhos/nota_empenho', '/v1/empenhos/processo', '/v1/empenhos/dotacao'])

In [10]:
def search_empenhos(val:str, tipo:str, ano=2022, mes=1):
    
    if type(val) != str:
        val = str(val)
    
    mapp_endpoint = {
        'nota_empenho' : '/v1/empenhos/nota_empenho',
        'processo' : '/v1/empenhos/processo',
        'dotacao' : '/v1/empenhos/dotacao'
    }
    
    if tipo not in mapp_endpoint:
        raise ValueError(f'Tipo must be in: {mapp_endpoint.keys()}')
        
    data = {
        tipo : val,
        'ano' : ano,
        'mes' : mes
    }
    print(data)
    return post_request(mapp_endpoint[tipo], data=data)

Teste de consistência

In [11]:
nota_empenho = "16830/2022"

In [12]:
resp_nota = search_empenhos(nota_empenho, 'nota_empenho', ano=2020, mes=8)

{'nota_empenho': '16830/2022', 'ano': 2020, 'mes': 8}


In [13]:
resp_nota

{'data': [{'empenho_liquido': 66343.61,
   'val_liquidado': 66343.61,
   'dotacao': '37.10.15.122.3024.2.100.33503900.00',
   'processo': 6066201600000670}],
 'metadados': {'sucess': True, 'message': None}}

In [14]:
proc = resp_nota['data'][0]['processo']

In [15]:
dotacao = resp_nota['data'][0]['dotacao']

In [16]:
proc

6066201600000670

In [17]:
resp_proc = search_empenhos(proc, 'processo', ano=2020, mes=8)

{'processo': '6066201600000670', 'ano': 2020, 'mes': 8}


In [18]:
dotacoes = set([item['dotacao'] for item in resp_proc['data']])

In [19]:
dotacoes

{'29.10.15.122.3024.2.100.33503900.00',
 '29.10.15.122.3024.2.100.33504800.00',
 '29.10.15.122.3024.2.100.33903900.00',
 '37.10.15.122.3024.2.100.33503900.00',
 '37.10.15.122.3024.2.100.33504800.00',
 '37.10.15.122.3024.2.100.33903900.00'}

In [20]:
assert dotacao in dotacoes

In [21]:
resp_dotacao = search_empenhos(dotacao, 'dotacao', ano=2020, mes=8)

{'dotacao': '37.10.15.122.3024.2.100.33503900.00', 'ano': 2020, 'mes': 8}


In [22]:
resp_dotacao

{'data': [{'empenho_liquido': 394057.5,
   'val_liquidado': 281459.77,
   'dotacao': '37.10.15.122.3024.2.100.33503900.00',
   'processo': None}],
 'metadados': {'sucess': True, 'message': None}}